# Variables

- memory references
- co doopravy jsou proměnné
- memory management => reference counting a garbage collection
- dynamický x statický typing
- mutabulity x immutability
- shared references
- rovnost proměnných
- VŠE JE OBJEKT

## Variables are memory references

Používáme paměť pro ukládání objektů (v P vše). Objekt může mít více adress - více slotů. Python memory manager se o tuto logiku stará sám - dolování dat.

my_var_1 = 10 => Python vytvoří v pamětí a zapíše 10  --> a my_var_1 je reference na paměť. Adresa ve formátu například 0x1000. Tzn. neznámá se nerovná neznámá ale pouze reference na paměť, kde se hodnota nalézá.

Pomocí funkce id() => zjistíme aresu v paměti. Navrací base-10 číslo.

In [1]:
my_var = 10

In [3]:
print(my_var) #my_var - adresa - cesta do paměti a přinese výsledek

10


In [6]:
print(id(my_var))
#naše adresa v paměti

140731916429360


In [9]:
print(hex(id(my_var)))
#hex verze naší adresy

0x7ffeb3e32830


In [10]:
greeting = "hello"

In [11]:
print(greeting)

hello


In [17]:
print(id(greeting))

2658293072944


In [18]:
print(hex(id(greeting)))

0x26aee9d4430


Tak tohle je poměrně jasné :)

## Reference Counting

Pokud mám my_var = 10, ale zadám other_var = my_var ---> nemám nový objekt mám stále stejnou adresu v paměti. Sdílí referenci a náš reference count = 2. Python Memory Manager má na starosti, ale vždy je o tom dobré vědět :).

In [20]:
#sys.getrefcount(my_var) ---> vytváří extra referenci, která nám má získat počet referencí
#, a tím náš count zvyšuje o jednu

In [21]:
# ctypes.c_long.from_adress(adress).value --> dáváme ale adresu paměti a zjišťujeme referenci
#posíláme mu naší přesnou adresu -> nic nezvyšujeme o 1

In [22]:
import sys

In [23]:
a = [1,2,3]

In [24]:
id(a)

2658292540288

In [26]:
sys.getrefcount(a) #a máme dva ikdyž máme jen jednu :) protože abychom zjistili kolik tam je referencí
#musíme poslat referenci

2

In [27]:
import ctypes

In [32]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value
#pouze funkce abychom nemuseli psát

In [34]:
ref_count(id(a)) #a nyní máme jen jednu
#nepředávám mu a - ale id(a) - už naší adresu

1

In [35]:
ref_count(2658292540288) #totožné jako id(a)

1

In [36]:
b = a 

In [38]:
id(b),id(a) #stejné adresy - pouze jsme předali referenci

(2658292540288, 2658292540288)

In [39]:
ref_count(id(a)), ref_count(id(b))

(2, 2)

In [49]:
c = a

In [50]:
ref_count(id(c))

3

Poměrně přímočaré. Jak se říká abychom pochopili co se děje under the hood. Přímo se nikdy o toto nesnažíme.

## Garbage Collection

Circular reference. Stav, kdy na sebe vzájemně odkazuje v kruhu více objektů. Tzn. pokud se zruší reference jednoho ale je zde propojení mezi dalšími. Reference counting začne selhávat a přijde na řade Garbage Collector. 

Garbage Collector je defaultně zapnout, můžeme jej vypnout ale pokud máme circular reference a vypnemem jej máme problém :). Proč bychom jej vypnuli? Abychom zvýšili výkon. Lepší je neřešit jej pokud není opravdu potřeba. Můžeme s ním provádět interakci, lze provádět i manulální cleaning.

Pro Python < 3.4 měl GC častější problémy pokud měl reference na objekty s destructorem __del__() => pro ničení objektu. 

In [54]:
import ctypes
import gc

In [55]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value

In [71]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(object) == object_id:
            return "Object exists"
    return "Not found"
#prohledá GC podle naší adresy a řekne nmám jestli v něm je nebo ne

In [59]:
class A:
    def __init__(self):
        self.b = B(self)
        print(f"A: self: {hex(id(self))}, b:{hex(id(self.b))}")
        #přepošleme A do classy B

In [58]:
class B:
    def __init__(self,a):
        #a je parametr který dáváme pro A při tvorbě
        self.a = a
        print(f"B: self: {hex(id(self))}, a:{hex(id(self.a))}")

Zde vytváříme dvě na sobě propojené classy. Kde jedna vzniká s druhou.

In [82]:
gc.disable() #musíme jej vypnout

In [61]:
my_var = A()

B: self: 0x26aeeaa4dc0, a:0x26aeeaa4280
A: self: 0x26aeeaa4280, b:0x26aeeaa4dc0


Mají propojené reference :)

In [62]:
hex(id(my_var)) #ukazuje na A - takže ok

'0x26aeeaa4280'

In [75]:
a_id = id(my_var)
b_id = id(my_var.b)

Trochu motání s adresami nebudu moc řešit. Ukládám si je, jak se jich zbavíme nebyli bychom schopni se na ně odkázat.

In [68]:
ref_count(a_id) #sedí 

2

In [69]:
ref_count(b_id) #vše je ok 

1

In [84]:
object_by_id(a_id)
#problém je že by měl podle všeho fungovat :D
#nebudeme hrotit

'Not found'

In [79]:
my_var = None #ničíme referenci tak

In [83]:
ref_count(a_id) #už jsme jen na jedné

1

In [85]:
gc.collect() #hmm to mi udělá možná větší bordel
#na tohle téma to chtělo asi čerstvý list

1163

In [86]:
object_by_id(a_id) #a ted po cleaningu to má vypadat takto

'Not found'

In [87]:
ref_count(a_id) #a já už jsem na 0

0

In [88]:
ref_count(b_id)

0

Mě to sice funguje, ale podle všeho si to dělá co chce. Je to prostě trolič. Mě to asi už funguje lépe než jemu :D. Prostě se od toho držet dál.

## Dynamic vs Static Typing

Java, C++, Swift jsou staticky psané. 

- String myVar = "hello"; --> dáváme data typ před proměnnou, proměnná je propojená s data typem
- U Pythona není neznámá propojená s data typem

In [95]:
a = 10
a, type(a)

(10, int)

In [96]:
a = "abc"
a

'abc'

V Pythonu to lze takto, pouze refrence na objekt a mohu změnit data typ. U statiscky typ nelze. Náš myvar nikdy neměl typ v Pythonu.

In [97]:
type(a)

str

## Variable RE-ASSIGNMENT

In [98]:
my_var = 10 # ----> referece na objekt který má data
my_var = 15 # ----> reference se přesouvá ale na nový objekt 
#STEJNÁ REFERENCE - ALE JINÝ OBJEKT
my_var = my_var +5 
#najde referenci na my_var a přidá +5 => vytvoří nový objekt
#a my_var předělá referenci na tento objekt

In [99]:
a = 10

In [100]:
hex(id(a))

'0x7ffeb3e32830'

In [101]:
type(a)

int

In [102]:
a = 15

In [103]:
hex(id(a)) #a vidíme že adresa objektu se změnila - přesunula se reference ale obejkt už je jinde v paměti

'0x7ffeb3e328d0'

In [104]:
a = a+1

In [106]:
hex(id(a)) #a též to samé

'0x7ffeb3e328f0'

Tohle je zajímavé :)

In [107]:
a = 10
b = 10

In [108]:
hex(id(a))

'0x7ffeb3e32830'

In [109]:
hex(id(b))

'0x7ffeb3e32830'

Co se tady děje? Za 1 vrátili jsme adresu z první a = 10 (tj spíše náhoda) ale máme pro a i b stejnou adresu. Možná se snaží být efektivní? Stejná hodnota, proč mít někde 2x když stačí jen dvě reference :).

## Object Mutability

my_account --->  reference 1000 ---> hodnota: 5 ---> změna na hodnotu: 10 -----> Reference zůstává stejná

Objekt je "mutable" tzn. lze změnit jeho hodnotu beze změny reference, immutable - nelze měnit

- Immutable - int,float,booleans -> nemůžeme měnit jejich interní hodnotu - tvoříme zde nový objekt - novou referenci
- Strings - jsou též immutable
- Tuples immutable
- Frozen Sets immutable
- User-Defined Classes - nastavíme si tak classu pokud chceme

Mutable

- Lists
- Sets
- Dictionaries
- User-Defined Classes - tzn. u Classy je to na mě

In [1]:
# t = (1,2,3) immutable
# a = [1,2] mmutable
# b = [3,4] mmutable
# t = (a,b) ===> t = ([1,2],[3,4]) máme ale muttable v immutable :))
# tzn t nezměním ale jeho a,b hodnoty měnit mohu

In [3]:
my_list = [1,2,3]
type(my_list) #id

list

In [4]:
my_list.append(4) #modifikuji data 

In [5]:
my_list #id zůstává stejné

[1, 2, 3, 4]

In [6]:
my_list_1 = [1,2,3] #adresa

In [7]:
my_list_1 = my_list_1 + [4] #adresa se mění nemám modifikaci dat

In [8]:
my_list_1 #tvořím nová data s novým id

[1, 2, 3, 4]

Tzn. my_list.append(4) - beze změny id, my_list_1 + [4] ---> změna id

In [9]:
my_dict = dict(key1=1,key2="a")
my_dict

{'key1': 1, 'key2': 'a'}

In [10]:
id(my_dict)

2237134422912

In [11]:
my_dict["key3"] = 10.5

In [12]:
my_dict

{'key1': 1, 'key2': 'a', 'key3': 10.5}

In [13]:
id(my_dict)

2237134422912

ID beze změny pouze modifikace

In [17]:
t = (1,2,3)

In [18]:
id(t)

2237134451648

In [21]:
id(t[0]), id(t[1]), id(t[2])
#id pro celek a id pro jednotlivé prvky

(140709391443728, 140709391443760, 140709391443792)

In [22]:
t = ([1,2], [3,4])

In [24]:
t[0], t[1]

([1, 2], [3, 4])

In [26]:
t[0].append(3)

In [27]:
t[0]

[1, 2, 3]

## Function Arguments and Mutability

In [28]:
# my_var = "hello" --immutable, nikdy nezměním pouze tak, že změním refrenci - nová data

Immutable hodnoty mají pojistku proti nechtěným operacím.

V podstatě pokud mám immutable hodnotu a mám například funkci, která jí modifikuje - zachováváme jak hodnotu a její referenci tak vytváříme i novou hodnotu a novou refrenci. Ale pokud se jedná například o list - dochází k modifikaci a tudíž náš list se změní. Například u našeho tuplu s listem též dojde ke změně beze změny adresy.

In [30]:
def process(s):
    print(f"my id is: {id(s)}")
    s = s + "world"
    print(f"Final s is: {id(s)}")

In [31]:
my_var = "hello"
print(f"my_var id is: {id(my_var)}")

my_var id is: 2237134489904


In [32]:
process(my_var)

my id is: 2237134489904
Final s is: 2237134487984


Takže co se tu děje - modifikujeme string. Jelikož string je mutable zacháváv své id. Ale naše modifikované s - již má nové id protože vznikl nový obejtk. Původní hodnota zachována a původní vytvořená.

In [33]:
def modify_list(lst):
    print(f"my id is: {id(lst)}")
    lst.append(100)
    print(f"Final s is: {id(lst)}")

In [34]:
my_list = [1,2,3]
id(my_list)

2237133149632

In [35]:
modify_list(my_list)

my id is: 2237133149632
Final s is: 2237133149632


U listu se žádné id nemění. Má mutable hodnota je změněná.

In [36]:
my_list

[1, 2, 3, 100]

Immutable mají pojistku a na muttable pozor. Snáže u nich ztratím jejich původní hodnotu.

In [38]:
def modify_tuple(t):
    print(f"my id is: {id(t)}")
    t[0].append(100)
    print(f"Final s is: {id(t)}")

In [39]:
t = ([1,2],"a")

In [40]:
modify_tuple(t)

my id is: 2237133150528
Final s is: 2237133150528


In [41]:
t

([1, 2, 100], 'a')

Takže zde mám tuple, který je immutable ale stejně můžu provézt modifikace listu uvnitř, který je mutable. A id zůstává stejný - měníme objekt.

## Shared references and mutability

Stav kdy dvě neznámé odkazují na stejný objekt v paměti.

In [42]:
a = 10
b = 10

In [43]:
id(a)

140709391444016

In [44]:
id(b)

140709391444016

Mám dvě neznámé ale jedno id. Automaticky b ukazuje na objekt a. Prostě je vyhodnotí jako shodné a usoudí, že je vhodné to tak použít. Právě proto, že jsou immutable tak při změně vzniká nový objekt a druhý i tak nadále existuje.

In [45]:
a=  [1,2,3]
b = a

In [46]:
id(a)

2237133614912

In [47]:
id(b)

2237133614912

Tak a zde musíme být opatrní :).

In [48]:
b.append(100)

In [51]:
a, b, id(a), id(b)

([1, 2, 3, 100], [1, 2, 3, 100], 2237133614912, 2237133614912)

Tzn. musím dávat bacha měním oba objekty protože mají stejný objekt. Pokud bych například b něco přiřadil tvořil bych nový objekt. Appendem jej ale modifikuji. Například pokud vkládám list do funkce - měním reference a funkce mi změní list mimo (což se mi stávalo pořád :) )

In [52]:
a = 500
b = 500

In [53]:
id(a)

2237134523056

In [54]:
id(b)

2237134522640

Tak a to je zajímavé. Né vždy využije možnost použít stejný "slot" pro shodná data. K tomu až dále.

## Variable Equality

Jak zjišťujeme?

- na úrovni paměti - memory adress
- object state(data)
- memory address => identify operater = is ..var1 is var2
- object data => identify operater = == :) ..var1 == var2
- tzn. shodu adresy zjištujeme pomocí is a shodu dat pomocí ==, a pomocí = máme assigment
- negace pro adresy var1 is not var2 nebo not(var1 is var2)
- negace pro data => var1 != var2 nebo not(var1 == var2)

In [55]:
a = 10
b = a

In [56]:
a is b, a == b

(True, True)

Mají stejnou hodnotu i adresu.

In [57]:
a = "hello"
b = "hello"

In [59]:
a is b, a == b

(True, True)

To samé a logické. Jenže samozřejmě adresa se může zbláznit a - a is b nebude sedět.

In [60]:
a = 500
b = 500

In [62]:
a is b # :D pecka prostě

False

In [63]:
a = [1,2,3]
b = [1,2,3]

In [65]:
a is b, a == b

(False, True)

Tohle je poměrně jasné. A a B jsem nijak nepropojil proto nemá adresové propojení. Musel bych uvést že a = b.

In [66]:
a = 10
b = 10.0

In [68]:
a is b, a == b

(False, True)

Python je chytrá opička a poradí si stím že 10 a 10.0 jsou shodné.

### None object

Prostě abychom indikovali že nemá danou hodnotu - ale je to objekt, který má shared referenci.

In [69]:
a = None
b = None
c = None

In [71]:
id(a), id(b), id(c)

(140709391210624, 140709391210624, 140709391210624)

Mají stejnou adresu, prostě náš objekt None :)

In [73]:
a is None, a == c

(True, True)

In [74]:
type(a)

NoneType

V podstatě jde o logiku mít None objekt, mohu tak ověřovat zda něco nabývá hodnoty či nikoliv. If it is none do that ..otherwise do other and so on..

## Everything is an object

Všechno jsou objekty - jsou to instance class. Například v pythonu - když voláme funkci ta volá classu která vytvoří objekt funkce. Tzn - všechno je objekt a všechno je classa a všechno má adresu v paměti.

In [75]:
def my_func(a,b):
    return a*b

In [76]:
id(my_func) #tzn. i funkce má adresu

2237134331328

- Každému může být přiřazena hodnota - třeba i funkcím
- Každému objektu může být "passed" funkce
- Každý objet může být navrácen z funkce
- V podstatě je to divokej cirkus :)

In [77]:
# my_func = name of the function, my_func() - invoke the function

In [78]:
a = 10

In [80]:
print(type(a)) #je to classa :)

<class 'int'>


In [82]:
#jelikož máme classu int - můžeme použít syntax pro tvorbu classy int a to takto:
b = int(10)

In [83]:
print(type(b))

<class 'int'>


In [85]:
#help(int) --> zobrazí info o classe int

In [88]:
c = int("101", base=2) #v binárním počtu - nemám tucha
c

5

### Teď k funkcím

In [89]:
def square(a):
    return a **2

In [91]:
print(type(square))

<class 'function'>


In [92]:
#jelikož naše square je objekt a funkce a má referenci mohu jí přiřadit
f = square 

In [93]:
id(square), id(f) #stejné 

(2237134331760, 2237134331760)

In [94]:
f is square

True

In [95]:
square(2), f(2)
#mohu teď zavolat obě funkce

(4, 4)

In [97]:
def cube(a):
    return a**3

In [99]:
def select_function(fn_id):
    if fn_id == 1:
        return square
    else:
        return cube
#funkce obdrží číslo - pokud bude jedna
#navrátí referenci na square
#jinak navrátí referenci na cube

In [100]:
f = select_function(1)
#jelikož jsme zavolal function 1 a ten invoke
#přiřadili pro f tak - f obdrží square referenci

In [101]:
f is square #a tudíž tohle dává smysl

True

In [102]:
f(2) #a mohu jí zavolat
#moje f má referenci na funkci square :)

4

In [103]:
f = select_function(2)

In [104]:
f is cube

True

In [105]:
f(2)

8

In [107]:
#nebo mohu takto
select_function(2)(3)
#takže moje select function se spustí - vybere se jako cube
#a projede cube funkci s parametrem pro 3

27

In [108]:
def exec_function(fn,n):
    return fn(n)

In [109]:
exec_function(cube,3)
#funkce přijmula funkci a provedla ji

27

Nicméně teď to začíná dávat poměrně větší smysl. Tzn. celá logika na reference a na objekty.

## Python optimizations

Máme víc Pythonů - my pravděpodobně CPython, ale máme například Jython(java), IronPython a Pypy apod..

a = 10
b = 10
c = 500
d = 500

Proč jedno má jednu referenci a druhý ne? CPython přednačítá pro tento účel pouze -5 až 256. Má je načtený předem - nejčastěji používaná čísla. Jedná se tedy o logiku optimalizace. V podstatě u těchto čísel ani nevytváří objekt ale jen  přiřadí k existujícímu.

In [ ]:
[-5,256]

In [110]:
a = 10

In [111]:
b = 10

In [112]:
id(a), id(b)

(140709391444016, 140709391444016)

In [113]:
a = -5
b = -5
id(a),id(b)

(140709391443536, 140709391443536)

In [115]:
a = -6
b = -6
id(a), id(b)
#Tadáá máme jiné id

(2237134556496, 2237134553968)

In [117]:
c = -6
d = c
id(c), id(d)
#takto je to samozřejmě ok

(2237134556336, 2237134556336)

In [119]:
c = int("10")
c
#chytrý to python 

10

## Python Optimizations - String Interning

Ne všechny stringy jsou "interned". Klasika je nepočítat stím :).

Pokud tedy vím, že mám interned stringy stačí mi porovnat pouze pomocí string is string. Místo logiky string == string. Je to mnohem rychlejší. Může ale manuálně donutit Python interned string. Pomocí sys.intern() metody. Znovu má to smysl pro rychlost.

Řekněme že mám hodně dat které disponují velkou opakovaností. Poté se to například hodí.

In [124]:
a = "hello"
b = "hello"
#budou internet - vypadají jako interned
id(a), id(b)
#a proto mají stejnou adresu

(2237134489904, 2237134489904)

In [127]:
a = "hello world"
b = "hello world"
id(a),id(b)
#nejsou interned - nevypadjaí jako identifikátor
#logicky se snaží držet v rychlosti to co je víc využívané

(2237134731952, 2237134732144)

In [128]:
a == b

True

In [130]:
a is b #zatím před  "hello" by mělo true :)

False

In [132]:
a = "_this_is_a_long_string_that_could_be_used_as_an_identifier"
b = "_this_is_a_long_string_that_could_be_used_as_an_identifier"

In [133]:
a is b #nejde o délku ale o to aby to vypadalo jako identifier :)

True

In [134]:
import sys

In [135]:
a = sys.intern("hello world")
b = sys.intern("hello world")
c = "hello world"

In [136]:
id(a),id(b),id(c)

(2237134741424, 2237134741424, 2237134741296)

Takže a a b jsou shodné. Manuálně jsem jim dopálil aby byli interned. C je sice stejný ale nemyslí si nebo mu nebylo řečeno že je to intern a tudíž má jinou adresu.

In [139]:
a == b, a == c, a is b, a is c
#jasné :)

(True, True, True, False)

Teď děláme náš speedtest :), prostě projede rovnosti a poté projede is díky tomu, že jsou interned.

In [141]:
def compare_using_equals(n):
        a = "a log string that is not interned" * 200
        b = "a log string that is not interned" * 200
        for i in range(n):
            if a == b:
                pass

In [142]:
def compare_using_interning(n):
        a = sys.intern("a log string that is not interned" * 200)
        b = sys.intern("a log string that is not interned" * 200)
        for i in range(n):
            if a is b:
                pass

In [143]:
import time

In [150]:
start = time.perf_counter()
compare_using_equals(1_0000_000)
end = time.perf_counter()
print(end-start)

3.1204810999988695


In [151]:
start = time.perf_counter()
compare_using_interning(1_0000_000)
end = time.perf_counter()
print(end-start)

0.5493744000013976


Takže vidíme výrazně lepší výsledek. Ať žije optimalizace.

## Python Optimizations PeepHole

- python vezme 24*60 a automaticky si zapamatuje, že se jedná o 1440, je pre-calculated a uloženo
- krátké sekvence <20 (1,2) *5 - pre-calculováno a uloženo
- "abc" *3 - pre calculated a uloženo
- Dělá to proto aby nemuselznovu a znovu vše přepočítávat, má tedy svoje uložiště toho co si nechá v sobě a to bude počítat. U Pythonu hranice kolem 20 znaků.


### Membership Tests: Mutable are raplaced by Immutables

In [152]:
#if e in [1,2,3]: --> pro python je [1,2,3] konstanta v tomto případě, nahradí to imutable částí 
#protože ví, že jej nebude muset měnit.

- listy mění na tuple 
- sets na frozensets

Set membership je výrazně rychlejší než listy a tuply, je mnohem rychlejší provádět "lookup".

In [153]:
# if e in [1,2,3]: --> list pomalý
# if e in (1,2,3): --> tuple pomalý
# if e in {1,2,3}: --> set rychlý 
#Myslet nad tím co používám pro lookup :)

In [156]:
def my_func():
    a = 24*60
    b = (1,2) *5
    c = "abc" *3
    d = "ab" *18
    e = "the quick brown fox" *5
    f = ["a","b"] *3

Python naší funkci s kompliloval.

In [157]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'abababababababababababababababababab',
 'the quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown fox',
 'a',
 'b',
 3)

Tak a zde jde vidět co si Python před-kalkuloval. V podstatš vše kromě poslední hodnoty.

In [158]:
def my_func(e):
    if e in [1,2,3]:
        pass

Tak ale Python si z [1,2,3] udělal tuple (1,2,3), aby to měl rychlejší.

In [159]:
my_func.__code__.co_consts

(None, (1, 2, 3))

In [160]:
def my_func(e):
    if e in {1,2,3}:
        pass

In [161]:
my_func.__code__.co_consts

(None, frozenset({1, 2, 3}))

Zde je převod na frozenset.

### Nyní k benchmarku

In [162]:
import string
import time

In [163]:
string.ascii_letters 

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [164]:
char_list = list(string.ascii_letters) 
char_tuple = tuple(string.ascii_letters)
char_set = set(string.ascii_letters)
#používám zkratky list, tuple apod..abych nemusel vypisovat postupně

In [166]:
char_list[:5]

['a', 'b', 'c', 'd', 'e']

In [169]:
print(char_tuple)

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')


In [170]:
print(char_set) #není seřazená sekvence jen pro zajímavost, tuple a list seřazeny

{'j', 'x', 'U', 'h', 'm', 'J', 'k', 's', 'q', 'w', 'F', 'u', 'e', 'n', 'T', 'l', 'Z', 'P', 'A', 'V', 'a', 'W', 'D', 'H', 'C', 'g', 'i', 'R', 'z', 'N', 'v', 'S', 'p', 'b', 'd', 'E', 'Y', 't', 'c', 'K', 'o', 'G', 'O', 'I', 'X', 'r', 'Q', 'M', 'B', 'L', 'y', 'f'}


In [171]:
def membership_test(n, container):
    for i in range(n):
        if "z" in container:
            pass

In [174]:
start = time.perf_counter()
membership_test(10_000_000, char_list)
end = time.perf_counter()
print(end-start)
#list má cca 4,2 sekundy

4.216523200000665


In [175]:
start = time.perf_counter()
membership_test(10_000_000, char_tuple)
end = time.perf_counter()
print(end-start)

4.289322299999185


In [ ]:
#žádný extra rozdíl

In [176]:
start = time.perf_counter()
membership_test(10_000_000, char_set)
end = time.perf_counter()
print(end-start)

0.6376985999995668


Set má teda výrazně rychlejší progress.